In [1]:
from Chess_Version_6 import Game, Custom_Game, Player, Board, square, Piece, Pawn, Queen, Rook, King, Knight, Bishop
import numpy as np
import math
import copy
#!pip install pygame
import pygame as p
W = H = 520
sq_size = W//8
imgs = {}
fpsmax = 15
import random

pygame 2.4.0 (SDL 2.26.4, Python 3.8.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
#We want to load in images to the game, but ideally we only want to do this once
def LoadingImg():
    pieces = ['P','R','B','Q','N','K','p','b','r','n','q','k','gd']
    for pc in pieces:
        col = 'W' if pc.upper() == pc else 'B'
        imgs[pc] = p.image.load('ChessImages/'+col+pc+'.png')
wood = p.image.load('ChessImages/background.jpg')
marble = p.image.load('ChessImages/marble.jpg')
sienna = p.image.load('ChessImages/sienna.jpg')

In [ ]:
# Click twice to move a piece

def main():
    
    frames = 15
    d = {'a':0,'b':1,'c':2,'d':3,'e':4,'f':5,'g':6,'h':7}
    
    p.init()
    screen = p.display.set_mode((W+60,H+120))
    clock = p.time.Clock()
    screen.fill(p.Color('white'))
    game = Game()
    LoadingImg()
    runs = True
    sq_selected = ()
    playerclicks = []
    
    input_box = p.Rect(188, 596, 140, 32)
    base_font = p.font.Font(None, 32)
    color_inactive = p.Color('lightskyblue3')
    color_active = p.Color('dodgerblue2')
    color = color_inactive
    active = False
    text = ''
    text2 = ''
    
    wst = p.time.get_ticks()
    
    mvs = None
    pos = None
    xm = ym = None
    player = random.choice([-1,1])
    
    while runs:
        
        ct = p.time.get_ticks()
        
        if ct < wst:
            
            text2 = ''
            screen.blit(win_surface, win_rect)
        
        for ev in p.event.get():
            
            if ev.type == p.QUIT:
                runs = False
                
            elif ev.type == p.MOUSEBUTTONDOWN:
                
                loc = p.mouse.get_pos()
                
                if input_box.collidepoint(ev.pos):
                    text = ''
                    # Toggle the active variable.
                    active = not active
                else:
                    active = False
                    # Change the current color of the input box.
                color = color_active if active else color_inactive
                
                if loc[1] <= 570 and loc[1] >= 50 and loc[0] >= 30:
                    
                    if player == 1:
                        x = (loc[0]-30)//sq_size
                        y = abs((loc[1]-50)//sq_size-7)
                        
                    elif player == -1:
                        x = (loc[0]-30)//sq_size
                        y = abs((loc[1]-50)//sq_size-7)
                        y = abs(y-7)
                        
                    print(loc,x,y)
                    
                    if game.ChessBoard.squares[y][x].occupied and not mvs:
                        
                        col,piece = game.ChessBoard.squares[y][x].occupied

                        mvs = game.ChessBoard.pieces[col][piece].get_moves()
                        #print(mvs)
                        
                        if piece.lower() == 'k':
                            for i in mvs:
                                
                                print(game.ChessBoard.squares[i[1]][i[0]].heat)
                        
                    if sq_selected == (x,y):

                        sq_selected = ()
                        playerclicks = []
                        mvs = None
                        pos = None

                    else:
                        
                        sq_selected = (x,y)
                        playerclicks.append(sq_selected)
                        pos = playerclicks[0]
                        
                    #### Testing print(len(playerclicks))

                    if len(playerclicks) == 2:

                        #print(playerclicks)
                        
                        xc,yc = playerclicks[0]
                        
                        if game.ChessBoard.squares[yc][xc].occupied:
                            
                            col,occ = game.ChessBoard.squares[yc][xc].occupied
                            pos = (xc,yc)
                            if occ[0].lower() == 'p':
                                
                                if y != 7 and y != 0:
                                    
                                    if col == game.ChessBoard.current_player.color:
                                        game.ChessBoard.pieces[col][occ].move_to((x,y))
                                        
                                    playerclicks = []
                                    sq_selected = ()
                                    mvs = None
                                    pos = None
                                    
                                else:
                                    xo,yo = playerclicks[0]
                                    xm,ym = playerclicks[1]
                                    playerclicks = []
                                    sq_selected = ()
                            else:
                                if col == game.ChessBoard.current_player.color:
                                        game.ChessBoard.pieces[col][occ].move_to((x,y))
                                        
                                playerclicks = []
                                sq_selected = ()
                                mvs = None
                                pos = None
                        
                        ### Testing print(game.ChessBoard.move_log)
                        
                    elif len(playerclicks) > 2:
                        
                        playerclicks = []
                        sq_selected = ()
                        mvs = None
                        pos = None
                        
            elif ev.type == p.KEYDOWN:
                    
                if active:
                    
                    if ev.key == p.K_RETURN:
                        
                        active = False
                        
                        move = text
                        
                        try:
            
                            if len(move) > 4:

                                prom = ''

                                for i in move:
                                    if i.isalpha():
                                        prom += i.lower()
                                prom = prom[0].upper()+prom[1:]

                            if prom:
                                m = (xm,ym,prom)
                                
                            ### Testing print(game.ChessBoard.squares[yc][xc].occupied,game.ChessBoard.current_player.color,mvs)

                            if game.ChessBoard.squares[yo][xo].occupied and prom:

                                col,piece = game.ChessBoard.squares[yo][xo].occupied

                                if col == game.ChessBoard.current_player.color:

                                    game.ChessBoard.pieces[col][piece].move_to(m)

                            mvs = None
                            pos = None
                            text = ''
                            
                        except:
                        
                            print("You Absolute BONOBO")
                            print('Try click and click')                        
                        
                    elif ev.key == p.K_BACKSPACE:
                        b = text[:-1]
                        text = b
                        
                        txt_surface = base_font.render(text, True, color)
                        width = max(200, txt_surface.get_width()+10)
                        input_box.w = width
                        # Blit the text.
                        screen.blit(txt_surface, (input_box.x+5, input_box.y+5))
                        # Blit the input_box rect.
                        #p.draw.rect(screen, color_inactive, (128, 522, 140, 32))
                        p.draw.rect(screen, color, input_box, 2)
                        
                        p.display.flip()
                        
                    else:
                        text += ev.unicode        
        
        if ct > wst:
            
            #screen.fill(p.Color('grey81'))
            screen.blit(wood, (0, 0))
            drawGS(screen,game,mvs,pos,player)
            
            txt_surface = base_font.render(text, True, color)
            width = max(200, txt_surface.get_width()+10)
            input_box.w = width
            # Blit the text.
            screen.blit(txt_surface, (input_box.x+5, input_box.y+5))
            # Blit the input_box rect.
            #p.draw.rect(screen, color_inactive, (128, 522, 140, 32))
            p.draw.rect(screen, color, input_box, 2)
        
        if game.ChessBoard.state():
            
            text2 = game.ChessBoard.state()[1]
            win_surface = base_font.render(str(text2).encode("utf-8"), True, p.Color('gold'))
            win_rect = win_surface.get_rect(center=((W+30)/2,(H+120)/2-8))
            #width = win_surface.get_width()+10
            #win_box.w = width
            # Blit the text.
            game = Game()         
            
            player = random.choice([1,-1])
            wst = ct + 10000
            
        clock.tick(fpsmax)
        p.display.flip()
        
def drawGS(screen,game,squares,pos,player):
    
    #Draws the game currently
    
    drawBoard(screen,squares,pos,player,game)
    drawPieces(screen,game.ChessBoard.update_board(),player)
    drawlabels(screen,player)
    
#Call the baord to draw first
def drawBoard(screen,squares,pos,player,game):
    
    if player == 1:
        func = lambda y: 50+33+abs(y-7)*sq_size
    elif player == -1:
        func = lambda y: 50+33+y*sq_size
        
    #cols = [marble,sienna]
    cols = [p.Color('seashell1'),p.Color('grey55')][::player]
    for c in range(8):
        for r in range(8):
            col = cols[(r+c) % 2]
            p.draw.rect(screen,col,p.Rect(30+c*sq_size,50+r*sq_size,sq_size,sq_size))
            #screen.blit(col,(30+c*sq_size,50+r*sq_size,sq_size,sq_size))
    if squares:
        for i in squares:
            if len(i) == 3:
                c,r,prom = i
            else:
                c,r = i
            xp,yp = c,r
            
            c,r = 63+xp*sq_size,func(yp)
            
            if game.ChessBoard.squares[yp][xp].occupied:
                for i in range(4):
                    p.draw.aaline(screen,"steelblue1",(c-33+15+i,r-33+14),(c+33-15+i,r+33-15))
                    p.draw.aaline(screen,"steelblue1",(c-33+15+i,r+33-15),(c+33-15+i,r-33+15)) 
                    p.draw.aaline(screen,"steelblue1",(c-33+15-i,r-33+15),(c+33-15-i,r+33-15))
                    p.draw.aaline(screen,"steelblue1",(c-33+15-i,r+33-15),(c+33-15-i,r-33+15)) 
                
                p.draw.ellipse(screen,"steelblue1",(c-33+4,r-33+4,sq_size-8,sq_size-8),width = 4)
                    
                
            else:
                
                #screen.blit(imgs['gd'],p.Rect(center = (30+c*sq_size/2,abs(r-7)*sq_size/2,4,4)))
                p.draw.circle(screen, p.Color('steelblue1'),(c,r),12)
                #p.draw.circle(screen,p.Color('grey79'),p.Rect(30+c*sq_size,abs(r-7)*sq_size,sq_size,sq_size))
    if pos:
        xp,yp = pos
        
        c,r = 30+xp*sq_size,func(yp)-33
            
        p.draw.rect(screen,p.Color('bisque'),p.Rect(c,r,sq_size,sq_size))        
    
def drawPieces(screen,piecelocs,player):
    
    bs = piecelocs[::-1*player]
    for c in range(8):
        for r in range(8):
            
            pcurr = bs[r][c]
            if pcurr != '*':
                screen.blit(imgs[pcurr[0]],p.Rect(30+1+c*sq_size,50+r*sq_size,sq_size,sq_size))
                
def drawlabels(screen,player):
    
    txt = p.font.Font(None, 32)
    
    lst = list('abcdefgh')
    
    if player == 1:
        func = lambda i: 528-(ind*65)
    elif player == -1:
        func = lambda i: 70+ind*65
    
    for ind,char in enumerate(lst):
        labcol = txt.render(char.encode("utf-8"),True,p.Color('white'))
        labrow = txt.render(str(ind+1).encode("utf-8"),True,p.Color('white'))
        screen.blit(labcol, (55+(ind*65), 572))
        screen.blit(labrow,(12,func(ind)))
            
if __name__ == '__main__':
    main()

(453, 70) 6 0
(395, 209) 5 2
(447, 538) 6 7
(380, 404) 5 5
(529, 68) 7 0
(529, 68) 7 0
(529, 68) 7 0
(529, 68) 7 0
(517, 83) 7 0
(517, 83) 7 0


In [3]:
# Click twice to move a piece

def main():
    p.init()
    screen = p.display.set_mode((W,H+40))
    clock = p.time.Clock()
    screen.fill(p.Color('white'))
    game = Game()
    LoadingImg()
    runs = True
    sq_selected = ()
    playerclicks = []
    
    input_box = p.Rect(128, 522, 140, 32)
    base_font = p.font.Font(None, 32)
    color_inactive = p.Color('lightskyblue3')
    color_active = p.Color('dodgerblue2')
    color = color_inactive
    active = False
    text = ''
    
    mvs = None
    
    while runs:
        
        for ev in p.event.get():
            
            if ev.type == p.QUIT:
                runs = False
                
            elif ev.type == p.MOUSEBUTTONDOWN:
                
                loc = p.mouse.get_pos()
                
                if loc[1] <= 512:
                    x = loc[0]//sq_size
                    y = abs(loc[1]//sq_size-7)

                    if game.ChessBoard.squares[y][x].occupied:

                        col,piece = game.ChessBoard.squares[y][x].occupied

                        mvs = game.ChessBoard.pieces[col][piece].get_moves()


                    if sq_selected == (x,y):

                        sq_selected = ()
                        playerclicks = []
                        mvs = None

                    else:

                        sq_selected = (x,y)
                        playerclicks.append(sq_selected)

                    print(len(playerclicks))

                    if len(playerclicks) == 2:

                        print(playerclicks)
                        xc,yc = playerclicks[0]
                        x,y = playerclicks[1]

                        colored,pc = game.ChessBoard.squares[yc][xc].occupied
                        w = (colored == 'White' and pc[0] == 'P' and y == 7)
                        b = (colored == 'Black' and pc[0] == 'p' and y == 0)

                        if w or b:

                            txt_surface = base_font.render(text, True, color)
                            width = max(200, txt_surface.get_width()+10)
                            input_box.w = width
                            # Blit the text.
                            screen.blit(txt_surface, (input_box.x+5, input_box.y+5))
                            # Blit the input_box rect.
                            p.draw.rect(screen, color, input_box, 2)

                        else:

                            print(game.ChessBoard.squares[yc][xc].occupied,game.ChessBoard.current_player.color,mvs)

                            if game.ChessBoard.squares[yc][xc].occupied:
                                col,piece = game.ChessBoard.squares[yc][xc].occupied

                                if col == game.ChessBoard.current_player.color:

                                    game.ChessBoard.pieces[col][piece].move_to((x,y))

                            playerclicks = []
                            sq_selected = ()
                            mvs = None

                    elif len(playerclicks) > 2:

                        playerclicks = []
                        sq_selected = ()
                    
            elif ev.type == p.KEYDOWN:
                    
                if active:
                    if ev.key == p.K_RETURN:
                        
                        prom = text

                        prom = None
                            
                        text = ''
                        #move = input('Please type in your move:')
                
                    elif ev.key == p.K_BACKSPACE:
                        text = text[:-1]
                        
                    else:
                        text += ev.unicode
                        
                        
        if game.ChessBoard.state():
                
            return game.ChessBoard.state()
                    
                
        drawGS(screen,game,mvs)
        
        clock.tick(fpsmax)
        p.display.flip()
        
def drawGS(screen,game,squares):
    
    #Draws the game currently
    
    drawBoard(screen,squares)
    drawPieces(screen,game.ChessBoard.update_board())

#Call the baord to draw first
def drawBoard(screen,squares):
    cols = [p.Color('white'),p.Color('grey')]
    for c in range(8):
        for r in range(8):
            col = cols[(r+c) % 2]
            p.draw.rect(screen,col,p.Rect(c*sq_size,r*sq_size,sq_size,sq_size))
            
    if squares:
        for i in squares:
            if len(i) == 3:
                c,r,prom = i
            else:
                c,r = i
                
            p.draw.rect(screen,p.Color('red'),p.Rect(c*sq_size,abs(r-7)*sq_size,sq_size,sq_size))

    
def drawPieces(screen,piecelocs):
    bs = piecelocs[::-1]
    for c in range(8):
        for r in range(8):
            
            pcurr = bs[r][c]
            if pcurr != '*':
                screen.blit(imgs[pcurr[0]],p.Rect(c*sq_size,r*sq_size,sq_size,sq_size))
        
if __name__ == '__main__':
    main()

1
2
[(3, 1), (3, 3)]
['White', 'P4'] White [(3, 2), (3, 3)]
1
2
[(3, 6), (3, 4)]
['Black', 'p4'] Black [(3, 4), (3, 5)]
1
2
[(4, 1), (4, 3)]
['White', 'P5'] White [(4, 2), (4, 3)]
1
2
[(4, 6), (4, 4)]
['Black', 'p5'] Black [(4, 4), (4, 5)]
1
2
[(3, 3), (4, 4)]
['White', 'P4'] White [(3, 3)]
1
2
[(5, 6), (5, 4)]
['Black', 'p6'] Black [(5, 4), (5, 5)]
1
2
[(4, 4), (5, 5)]
['White', 'P4'] White [(4, 5), (5, 5)]
1
2
[(7, 6), (7, 4)]
['Black', 'p8'] Black [(7, 4), (7, 5)]
1
2
[(5, 5), (6, 6)]
['White', 'P4'] White [(5, 5), (6, 4), (6, 5)]
1
2
[(5, 7), (2, 4)]
['Black', 'bk'] Black [(2, 4), (4, 6), (0, 2), (6, 6), (1, 3), (3, 5)]
1
2
[(6, 6), (7, 7)]
3
1


In [3]:
# Version 1, simply click once for the piece and moves, click again to move the piece

def main():
    p.init()
    screen = p.display.set_mode((W,H))
    clock = p.time.Clock()
    screen.fill(p.Color('white'))
    game = Game()
    LoadingImg()
    runs = True
    sq_selected = ()
    playerclicks = []
    
    mvs = None
    
    while runs:
        
        for ev in p.event.get():
            
            if ev.type == p.QUIT:
                runs = False
                
            elif ev.type == p.MOUSEBUTTONDOWN:
                
                loc = p.mouse.get_pos()
                
                x = loc[0]//sq_size
                y = abs(loc[1]//sq_size-7)
                
                if game.ChessBoard.squares[y][x].occupied:

                    col,piece = game.ChessBoard.squares[y][x].occupied

                    mvs = game.ChessBoard.pieces[col][piece].get_moves()
                
                
                if sq_selected == (x,y):
                    
                    sq_selected = ()
                    playerclicks = []
                    mvs = None
                    
                else:
                    
                    sq_selected = (x,y)
                    playerclicks.append(sq_selected)

                print(len(playerclicks))
                
                if len(playerclicks) == 2:
                                        
                    print(playerclicks)
                    xc,yc = playerclicks[0]
                    x,y = playerclicks[1]
                        
                    print(game.ChessBoard.squares[yc][xc].occupied,game.ChessBoard.current_player.color,mvs)
                    
                    if game.ChessBoard.squares[yc][xc].occupied:
                        col,piece = game.ChessBoard.squares[yc][xc].occupied
                        
                        if col == game.ChessBoard.current_player.color:
                        
                            game.ChessBoard.pieces[col][piece].move_to(m)

                    playerclicks = []
                    sq_selected = ()
                    mvs = None
                    
                elif len(playerclicks) > 2:
                    
                    playerclicks = []
                    sq_selected = ()
                    
        if game.ChessBoard.state():
                
            return game.ChessBoard.state()
                    
                
        drawGS(screen,game,mvs)
    
        clock.tick(fpsmax)
        p.display.flip()
        
def drawGS(screen,game,squares):
    
    #Draws the game currently
    
    drawBoard(screen,squares)
    drawPieces(screen,game.ChessBoard.update_board())

#Call the baord to draw first
def drawBoard(screen,squares):
    cols = [p.Color('white'),p.Color('grey')]
    for c in range(8):
        for r in range(8):
            col = cols[(r+c) % 2]
            p.draw.rect(screen,col,p.Rect(c*sq_size,r*sq_size,sq_size,sq_size))
            
    if squares:
        for i in squares:
            if len(i) == 3:
                c,r,prom = i
            else:
                c,r = i
                
            p.draw.rect(screen,p.Color('red'),p.Rect(c*sq_size,abs(r-7)*sq_size,sq_size,sq_size))

    
def drawPieces(screen,piecelocs):
    bs = piecelocs[::-1]
    for c in range(8):
        for r in range(8):
            
            pcurr = bs[r][c]
            if pcurr != '*':
                screen.blit(imgs[pcurr[0]],p.Rect(c*sq_size,r*sq_size,sq_size,sq_size))
        
if __name__ == '__main__':
    main()

1
2
[(3, 1), (3, 3)]
['White', 'P4'] White [(3, 2), (3, 3)]


NameError: name 'm' is not defined

In [ ]:
## Version 2 Where the textbox is in the python console

def main():
    p.init()
    screen = p.display.set_mode((W,H))
    clock = p.time.Clock()
    screen.fill(p.Color('white'))
    game = Game()
    LoadingImg()
    runs = True
    sq_selected = ()
    playerclicks = []
    
    mvs = None
    
    while runs:
        
        for ev in p.event.get():
            
            if ev.type == p.QUIT:
                runs = False
                
            elif ev.type == p.MOUSEBUTTONDOWN:
                
                loc = p.mouse.get_pos()
                
                x = loc[0]//sq_size
                y = abs(loc[1]//sq_size-7)
                
                if game.ChessBoard.squares[y][x].occupied:

                    col,piece = game.ChessBoard.squares[y][x].occupied

                    mvs = game.ChessBoard.pieces[col][piece].get_moves()
                
                
                if sq_selected == (x,y):
                    
                    sq_selected = ()
                    playerclicks = []
                    mvs = None
                    
                else:
                    
                    sq_selected = (x,y)
                    playerclicks.append(sq_selected)

                print(len(playerclicks))
                
                if len(playerclicks) == 2:
                                        
                    print(playerclicks)
                    xc,yc = playerclicks[0]
                    x,y = playerclicks[1]
                    
                    move = input('Please type in your move:')
                    xm,ym = int(move[1]),int(move[3])
                    
                    prom = None
                    
                    if len(move) > 5:
                        prom = move[5:-1]
                        
                    if prom:
                        m = (xm,ym,prom)
                    
                    else:
                        m = (xm,ym)
                        
                    print(game.ChessBoard.squares[yc][xc].occupied,game.ChessBoard.current_player.color,mvs)
                    
                    if game.ChessBoard.squares[yc][xc].occupied:
                        col,piece = game.ChessBoard.squares[yc][xc].occupied
                        
                        if col == game.ChessBoard.current_player.color:
                        
                            game.ChessBoard.pieces[col][piece].move_to(m)

                    playerclicks = []
                    sq_selected = ()
                    mvs = None
                    
                elif len(playerclicks) > 2:
                    
                    playerclicks = []
                    sq_selected = ()
                    
        if game.ChessBoard.state():
                
            return game.ChessBoard.state()
                    
                
        drawGS(screen,game,mvs)
    
        clock.tick(fpsmax)
        p.display.flip()
        
def drawGS(screen,game,squares):
    
    #Draws the game currently
    
    drawBoard(screen,squares)
    drawPieces(screen,game.ChessBoard.update_board())

#Call the baord to draw first
def drawBoard(screen,squares):
    cols = [p.Color('white'),p.Color('grey')]
    for c in range(8):
        for r in range(8):
            col = cols[(r+c) % 2]
            p.draw.rect(screen,col,p.Rect(c*sq_size,r*sq_size,sq_size,sq_size))
            
    if squares:
        for i in squares:
            if len(i) == 3:
                c,r,prom = i
            else:
                c,r = i
                
            p.draw.rect(screen,p.Color('red'),p.Rect(c*sq_size,abs(r-7)*sq_size,sq_size,sq_size))

    
def drawPieces(screen,piecelocs):
    bs = piecelocs[::-1]
    for c in range(8):
        for r in range(8):
            
            pcurr = bs[r][c]
            if pcurr != '*':
                screen.blit(imgs[pcurr[0]],p.Rect(c*sq_size,r*sq_size,sq_size,sq_size))
        
if __name__ == '__main__':
    main()

In [ ]:
# Version 3 (text box input for moves instead)

def main():
    p.init()
    screen = p.display.set_mode((W,H+60))
    clock = p.time.Clock()
    screen.fill(p.Color('white'))
    game = Game()
    LoadingImg()
    runs = True
    sq_selected = ()
    playerclicks = []
    
    input_box = p.Rect(128, 522, 140, 32)
    base_font = p.font.Font(None, 32)
    color_inactive = p.Color('lightskyblue3')
    color_active = p.Color('dodgerblue2')
    color = color_inactive
    active = False
    text = ''
    
    mvs = None
    m = ()
    while runs:
        
        for ev in p.event.get():
            
            if ev.type == p.QUIT:
                runs = False
                
            elif ev.type == p.MOUSEBUTTONDOWN:
                
                if input_box.collidepoint(ev.pos):
                    # Toggle the active variable.
                    active = not active
                else:
                    active = False
                # Change the current color of the input box.
                color = color_active if active else color_inactive
                
                
                
                loc = p.mouse.get_pos()
                
                if loc[1] <= 512:
                
                    x = loc[0]//sq_size
                    y = abs(loc[1]//sq_size-7)
                
                    if game.ChessBoard.squares[y][x].occupied:

                        col,piece = game.ChessBoard.squares[y][x].occupied

                        mvs = game.ChessBoard.pieces[col][piece].get_moves()
                
                
                    if sq_selected == (x,y):

                        sq_selected = ()
                        playerclicks = []
                        mvs = None
                    
                    else:
                        
                        sq_selected  = (x,y)
                        playerclicks.append(x,y)
                        
                        sq_selected != (x,y) and len(m) > 0:

                        xc,yc = sq_selected
                        sq_selected = (x,y)

                        print(game.ChessBoard.squares[yc][xc].occupied,game.ChessBoard.current_player.color,mvs)

                        if game.ChessBoard.squares[yc][xc].occupied:
                            col,piece = game.ChessBoard.squares[yc][xc].occupied

                            if col == game.ChessBoard.current_player.color:

                                game.ChessBoard.pieces[col][piece].move_to(m)


                            playerclicks = []
                            sq_selected = ()
                            mvs = None
                    
                if len(playerclicks) > 1:
                    
                    playerclicks = playerclicks[-1]
                    sq_selected = ()
                    
            if ev.type == p.KEYDOWN:
                    
                if active:
                    if ev.key == p.K_RETURN and len(text) > 0:
                        
                        xm,ym = int(text[1]),int(text[3])

                        prom = None

                        if len(text) > 5:
                            prom = text[5:-1]

                        if prom:
                            m = (xm,ym,prom)

                        else:
                            m = (xm,ym)
                            
                        text = ''
                        #move = input('Please type in your move:')
                
                    elif ev.key == p.K_BACKSPACE:
                        text = text[:-1]
                        
                    else:
                        text += ev.unicode
        
        if game.ChessBoard.state():
                
            return game.ChessBoard.state()
                    
        
        drawGS(screen,game,mvs)
        txt_surface = base_font.render(text, True, color)
        width = max(200, txt_surface.get_width()+10)
        input_box.w = width
        # Blit the text.
        screen.blit(txt_surface, (input_box.x+5, input_box.y+5))
        # Blit the input_box rect.
        p.draw.rect(screen, color, input_box, 2)
        clock.tick(fpsmax)
        p.display.flip()
        
def drawGS(screen,game,squares):
    
    #Draws the game currently
    
    drawBoard(screen,squares)
    drawPieces(screen,game.ChessBoard.update_board())

#Call the baord to draw first
def drawBoard(screen,squares):
    cols = [p.Color('white'),p.Color('grey')]
    for c in range(8):
        for r in range(8):
            col = cols[(r+c) % 2]
            p.draw.rect(screen,col,p.Rect(c*sq_size,r*sq_size,sq_size,sq_size))
            
    if squares:
        for i in squares:
            if len(i) == 3:
                c,r,prom = i
            else:
                c,r = i
            p.draw.rect(screen,p.Color('red'),p.Rect(c*sq_size,abs(r-7)*sq_size,sq_size,sq_size))

    
def drawPieces(screen,piecelocs):
    bs = piecelocs[::-1]
    for c in range(8):
        for r in range(8):
            
            pcurr = bs[r][c]
            if pcurr != '*':
                screen.blit(imgs[pcurr[0]],p.Rect(c*sq_size,r*sq_size,sq_size,sq_size))
        
if __name__ == '__main__':
    main()

In [4]:
game = Game()
game.ChessBoard.current_player.pieces['P4'].move_to((3,3))
game.ChessBoard.update_board()

[['RQ', 'NQ', 'BQ', 'Q', 'K', 'BK', 'NK', 'RK'],
 ['P1', 'P2', 'P3', '*', 'P5', 'P6', 'P7', 'P8'],
 ['*', '*', '*', '*', '*', '*', '*', '*'],
 ['*', '*', '*', 'P4', '*', '*', '*', '*'],
 ['*', '*', '*', '*', '*', '*', '*', '*'],
 ['*', '*', '*', '*', '*', '*', '*', '*'],
 ['p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8'],
 ['rq', 'nq', 'bq', 'q', 'k', 'bk', 'nk', 'rk']]

In [5]:
g = input('Please type in your move:')
print(type(g))

Please type in your move:(3,3)
<class 'str'>


In [5]:
game.ChessBoard.current_player.pieces['p4'].move_to((3,4))
game.ChessBoard.update_board()

[['RQ', 'NQ', 'BQ', 'Q', 'K', 'BK', 'NK', 'RK'],
 ['P1', 'P2', 'P3', '*', 'P5', 'P6', 'P7', 'P8'],
 ['*', '*', '*', '*', '*', '*', '*', '*'],
 ['*', '*', '*', 'P4', '*', '*', '*', '*'],
 ['*', '*', '*', 'p4', '*', '*', '*', '*'],
 ['*', '*', '*', '*', '*', '*', '*', '*'],
 ['p1', 'p2', 'p3', '*', 'p5', 'p6', 'p7', 'p8'],
 ['rq', 'nq', 'bq', 'q', 'k', 'bk', 'nk', 'rk']]

In [6]:
game.ChessBoard.current_player.pieces['P5'].move_to((4,3))
game.ChessBoard.update_board()

[['RQ', 'NQ', 'BQ', 'Q', 'K', 'BK', 'NK', 'RK'],
 ['P1', 'P2', 'P3', '*', '*', 'P6', 'P7', 'P8'],
 ['*', '*', '*', '*', '*', '*', '*', '*'],
 ['*', '*', '*', 'P4', 'P5', '*', '*', '*'],
 ['*', '*', '*', 'p4', '*', '*', '*', '*'],
 ['*', '*', '*', '*', '*', '*', '*', '*'],
 ['p1', 'p2', 'p3', '*', 'p5', 'p6', 'p7', 'p8'],
 ['rq', 'nq', 'bq', 'q', 'k', 'bk', 'nk', 'rk']]

In [7]:
game.ChessBoard.current_player.pieces['p4'].move_to((4,3))
game.ChessBoard.update_board()

[['RQ', 'NQ', 'BQ', 'Q', 'K', 'BK', 'NK', 'RK'],
 ['P1', 'P2', 'P3', '*', '*', 'P6', 'P7', 'P8'],
 ['*', '*', '*', '*', '*', '*', '*', '*'],
 ['*', '*', '*', 'P4', 'p4', '*', '*', '*'],
 ['*', '*', '*', '*', '*', '*', '*', '*'],
 ['*', '*', '*', '*', '*', '*', '*', '*'],
 ['p1', 'p2', 'p3', '*', 'p5', 'p6', 'p7', 'p8'],
 ['rq', 'nq', 'bq', 'q', 'k', 'bk', 'nk', 'rk']]

In [5]:
def b(curr):
    x0,y0 = curr
    for i in range(y0-1,-1,-1):
        if (x0 in range(8)) and (i in range(8)):
            yield (x0,i)
        else:
            yield ()
    yield ()
    
a = b((2,1))

In [6]:
next(a)

(2, 0)

In [7]:
next(a)

()